In [2]:
!pip install rake_nltk

  Created wheel for rake-nltk: filename=rake_nltk-1.0.4-py2.py3-none-any.whl size=7818 sha256=2f20b8e406190163857fe6146ca068b00160d5b8c332edb47d490fa10c8efa75
  Stored in directory: /root/.cache/pip/wheels/ef/92/fc/271b3709e71a96ffe934b27818946b795ac6b9b8ff8682483f
Successfully built rake-nltk


In [19]:
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

pd.set_option('display.max_columns', 100)
df = pd.read_csv('https://query.data.world/s/uikepcpffyo2nhig52xxeevdialfl7')
df.head()

Unnamed: 0                     Title  Year     Rated     Released  Runtime  \
0           1  The Shawshank Redemption  1994         R  14 Oct 1994  142 min   
1           2             The Godfather  1972         R  24 Mar 1972  175 min   
2           3    The Godfather: Part II  1974         R  20 Dec 1974  202 min   
3           4           The Dark Knight  2008     PG-13  18 Jul 2008  152 min   
4           5              12 Angry Men  1957  APPROVED  01 Apr 1957   96 min   

                  Genre              Director  \
0          Crime, Drama        Frank Darabont   
1          Crime, Drama  Francis Ford Coppola   
2          Crime, Drama  Francis Ford Coppola   
3  Action, Crime, Drama     Christopher Nolan   
4          Crime, Drama          Sidney Lumet   

                                              Writer  \
0  Stephen King (short story "Rita Hayworth and S...   
1  Mario Puzo (screenplay), Francis Ford Coppola ...   
2  Francis Ford Coppola (screenplay), Mario Puzo ...   
3  Jonathan Nolan (screenplay), Christopher Nolan...   
4  Reginald Rose (story), Reginald Rose (screenplay)   

                                              Actors  \
0  Tim Robbins, Morgan Freeman, Bob Gunton, Willi...   
1  Marlon Brando, Al Pacino, James Caan, Richard ...   
2  Al Pacino, Robert Duvall, Diane Keaton, Robert...   
3  Christian Bale, Heath Ledger, Aaron Eckhart, M...   
4  Martin Balsam, John Fiedler, Lee J. Cobb, E.G....   

                                                Plot  \
0  Two imprisoned men bond over a number of years...   
1  The aging patriarch of an organized crime dyna...   
2  The early life and career of Vito Corleone in ...   
3  When the menace known as the Joker emerges fro...   
4  A jury holdout attempts to prevent a miscarria...   

                                     Language  Country  \
0                                     English      USA   
1                     English, Italian, Latin      USA   
2  English, Italian, Spanish, Latin, Sicilian      USA   
3                           English, Mandarin  USA, UK   
4                                     English      USA   

                                              Awards  \
0  Nominated for 7 Oscars. Another 19 wins & 30 n...   
1    Won 3 Oscars. Another 23 wins & 27 nominations.   
2    Won 6 Oscars. Another 10 wins & 20 nominations.   
3  Won 2 Oscars. Another 151 wins & 153 nominations.   
4  Nominated for 3 Oscars. Another 16 wins & 8 no...   

                                              Poster           Ratings.Source  \
0  https://images-na.ssl-images-amazon.com/images...  Internet Movie Database   
1  https://images-na.ssl-images-amazon.com/images...  Internet Movie Database   
2  https://images-na.ssl-images-amazon.com/images...  Internet Movie Database   
3  https://images-na.ssl-images-amazon.com/images...  Internet Movie Database   
4  https://images-na.ssl-images-amazon.com/images...  Internet Movie Database   

  Ratings.Value  Metascore  imdbRating  imdbVotes     imdbID   Type  \
0        9.3/10       80.0         9.3  1,825,626  tt0111161  movie   
1        9.2/10      100.0         9.2  1,243,444  tt0068646  movie   
2        9.0/10       85.0         9.0    856,870  tt0071562  movie   
3        9.0/10       82.0         9.0  1,802,351  tt0468569  movie   
4        8.9/10       96.0         8.9    494,215  tt0050083  movie   

   tomatoMeter  tomatoImage  tomatoRating  tomatoReviews  tomatoFresh  \
0          NaN          NaN           NaN            NaN          NaN   
1          NaN          NaN           NaN            NaN          NaN   
2          NaN          NaN           NaN            NaN          NaN   
3          NaN          NaN           NaN            NaN          NaN   
4          NaN          NaN           NaN            NaN          NaN   

   tomatoRotten  tomatoConsensus  tomatoUserMeter  tomatoUserRating  \
0           NaN              NaN              NaN               NaN   
1           NaN              

In [20]:
df = df[['Title','Genre','Director','Actors','Plot']]
df.head()

Title                 Genre              Director  \
0  The Shawshank Redemption          Crime, Drama        Frank Darabont   
1             The Godfather          Crime, Drama  Francis Ford Coppola   
2    The Godfather: Part II          Crime, Drama  Francis Ford Coppola   
3           The Dark Knight  Action, Crime, Drama     Christopher Nolan   
4              12 Angry Men          Crime, Drama          Sidney Lumet   

                                              Actors  \
0  Tim Robbins, Morgan Freeman, Bob Gunton, Willi...   
1  Marlon Brando, Al Pacino, James Caan, Richard ...   
2  Al Pacino, Robert Duvall, Diane Keaton, Robert...   
3  Christian Bale, Heath Ledger, Aaron Eckhart, M...   
4  Martin Balsam, John Fiedler, Lee J. Cobb, E.G....   

                                                Plot  
0  Two imprisoned men bond over a number of years...  
1  The aging patriarch of an organized crime dyna...  
2  The early life and career of Vito Corleone in ...  
3  When the menace known as the Joker emerges fro...  
4  A jury holdout attempts to prevent a miscarria...

In [0]:
df['Actors'] = df['Actors'].map(lambda x: x.split(',')[:3])


df['Genre'] = df['Genre'].map(lambda x: x.lower().split(','))

df['Director'] = df['Director'].map(lambda x: x.split(' '))


for index, row in df.iterrows():
    row['Actors'] = [x.lower().replace(' ','') for x in row['Actors']]
    row['Director'] = ''.join(row['Director']).lower()

In [0]:
df['Key_words'] = ""

for index, row in df.iterrows():
    plot = row['Plot']
    
    
    r = Rake()

    r.extract_keywords_from_text(plot)

    key_words_dict_scores = r.get_word_degrees()
    
    row['Key_words'] = list(key_words_dict_scores.keys())

df.drop(columns = ['Plot'], inplace = True)

In [23]:
df.set_index('Title', inplace = True)
df.head()

Genre            Director  \
Title                                                                    
The Shawshank Redemption           [crime,  drama]       frankdarabont   
The Godfather                      [crime,  drama]  francisfordcoppola   
The Godfather: Part II             [crime,  drama]  francisfordcoppola   
The Dark Knight           [action,  crime,  drama]    christophernolan   
12 Angry Men                       [crime,  drama]         sidneylumet   

                                                              Actors  \
Title                                                                  
The Shawshank Redemption      [timrobbins, morganfreeman, bobgunton]   
The Godfather                    [marlonbrando, alpacino, jamescaan]   
The Godfather: Part II         [alpacino, robertduvall, dianekeaton]   
The Dark Knight           [christianbale, heathledger, aaroneckhart]   
12 Angry Men                  [martinbalsam, johnfiedler, leej.cobb]   

                                                                  Key_words  
Title                                                                        
The Shawshank Redemption  [number, finding, solace, two, imprisoned, men...  
The Godfather             [organized, crime, dynasty, transfers, control...  
The Godfather: Part II    [1920s, new, york, expands, son, early, life, ...  
The Dark Knight           [menace, known, people, wreaks, havoc, greates...  
12 Angry Men              [justice, prevent, evidence, jury, holdout, at...

In [0]:
df['bag_of_words'] = ''
columns = df.columns
for index, row in df.iterrows():
    words = ''
    for col in columns:
        if col != 'Director':
            words = words + ' '.join(row[col])+ ' '
        else:
            words = words + row[col]+ ' '
    row['bag_of_words'] = words
    
df.drop(columns = [col for col in df.columns if col!= 'bag_of_words'], inplace = True)

In [25]:
df.head()

bag_of_words
Title                                                                      
The Shawshank Redemption  crime  drama frankdarabont timrobbins morganfr...
The Godfather             crime  drama francisfordcoppola marlonbrando a...
The Godfather: Part II    crime  drama francisfordcoppola alpacino rober...
The Dark Knight           action  crime  drama christophernolan christia...
12 Angry Men              crime  drama sidneylumet martinbalsam johnfied...

In [27]:
count = CountVectorizer()
count_matrix = count.fit_transform(df['bag_of_words'])

indices = pd.Series(df.index)
indices[:5]

0    The Shawshank Redemption
1               The Godfather
2      The Godfather: Part II
3             The Dark Knight
4                12 Angry Men
Name: Title, dtype: object

In [28]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.15789474, 0.13764944, ..., 0.05263158, 0.05263158,
        0.05564149],
       [0.15789474, 1.        , 0.36706517, ..., 0.05263158, 0.05263158,
        0.05564149],
       [0.13764944, 0.36706517, 1.        , ..., 0.04588315, 0.04588315,
        0.04850713],
       ...,
       [0.05263158, 0.05263158, 0.04588315, ..., 1.        , 0.05263158,
        0.05564149],
       [0.05263158, 0.05263158, 0.04588315, ..., 0.05263158, 1.        ,
        0.05564149],
       [0.05564149, 0.05564149, 0.04850713, ..., 0.05564149, 0.05564149,
        1.        ]])

In [0]:
def recommendations(title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    
    idx = indices[indices == title].index[0]

    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    top_10_indexes = list(score_series.iloc[1:11].index)
    
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

In [41]:
recommendations('Inception')

['Interstellar',
 'Aliens',
 'Guardians of the Galaxy',
 'Guardians of the Galaxy Vol. 2',
 'Mad Max: Fury Road',
 'Spider-Man: Homecoming',
 'The Prestige',
 'The Matrix',
 'Batman Begins',
 '2001: A Space Odyssey']